In [1]:
!pip3 install -q cassio datasets langchain openai tiktoken


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip3 install langchain-community langchain-core


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
#necessary langchain components

from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings


#cassio will be used to integrate database in the langchain
import cassio

In [4]:
!pip3 install PyPDF2             #it is required to read the pdf 

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
from PyPDF2 import PdfReader

In [6]:
#necessary database token, id and openai_api key


ASTRA_DB_APPLICATION_TOKEN= # put the database token
ASTRA_DB_ID= # put the database id
OPENAI_API_KEY= #put openai_api_key

In [7]:
#it is the pdf from where the question will be asked 

pdfreader=PdfReader("/home/sucheta/Downloads/GraMMY.pdf")  

In [8]:
#raw_text contains the full text from the input pdf


from typing_extensions import Concatenate
raw_text=""
for i, page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content       

In [9]:
print(raw_text[:100])           #it will print first 100 character from the pdf

GraMMy: Graph Representation Learning based on
Micro-Macro Analysis
Sucheta Dawn, Monidipa Das, Sang


In [11]:
#initialization of connection between database and langchain 
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [12]:
#llm object and langchain embedding

llm=OpenAI(openai_api_key=OPENAI_API_KEY)
embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/home/sucheta/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/home/sucheta/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
# langchain vector store will be created in the database in the table named as "QA_from_GraMMy"
astra_vector_store=Cassandra(
embedding=embedding,
table_name="QA_from_GRaMMy",
session=None,
keyspace=None
)

In [14]:
#the whole text will be splitted into text chunk of chunk size 800


from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(
separator="\n",
chunk_size=800,
chunk_overlap=200,
length_function=len,
)
texts=text_splitter.split_text(raw_text)

In [15]:
#showsfirst 10 text chunks

texts[:10]

['GraMMy: Graph Representation Learning based on\nMicro-Macro Analysis\nSucheta Dawn, Monidipa Das, Sanghamitra Bandyopadhyay\nMachine Intelligence Unit, Indian Statistical Institute, Kolkata, India\nAbstract\nGraph Neural Networks (GNNs) are robust variants of deep network models, typ-\nically designed to learn from graph-structured data. Despite the recent advance-\nment of GNNs, the basic message passing scheme of learning often holds back\nthese models in effectively capturing the inﬂuence of the nodes from higher order\nneighbourhood. Further, the state-of-the-art approaches mostly ignore the contex-\ntual signiﬁcance of the paths through which the message/information propagates\nto a node. In order to deal with these two issues, we propose GraMMY as a novel',
 'tual signiﬁcance of the paths through which the message/information propagates\nto a node. In order to deal with these two issues, we propose GraMMY as a novel\nframework for hierarchical semantics-driven gra ph representa

In [16]:
astra_vector_store.add_texts(texts)
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [18]:
questionreader=PdfReader("/home/sucheta/Downloads/Question.pdf")
raw_question=""
for i, page in enumerate(questionreader.pages):
    content=page.extract_text()
    if content:
        raw_question+=content

In [19]:
print(len(raw_question))

100


In [20]:
questions=raw_question.split("\n")
print(questions)
print(len(questions))

['What is Micro-macro scalar?', 'What is the first paper in the Reference?', 'Who are the authors of GraMMy?']
3


In [21]:
with open('Answer.txt','w') as g:
    for question in questions:
        g.write("Question: "+str(question)+"\n")
        print(question)
        answer=astra_vector_index.query(question,llm=llm).strip()
        for doc, score in astra_vector_store.similarity_search_with_score(question,k=1):
            print(score)
            if(score<0.80):
                print("Data not available")
                g.write("Answer: "+"Data not available"+"\n")
            else:
                print("Answer%s"%answer)
                g.write("Answer: "+str(answer)+"\n")

What is Micro-macro scalar?
0.913121013542352
AnswerMicro-macro scalar is a function that takes a graph G and an amount of detail f as inputs and outputs another graph G0f and an epimorphismf such that it decides the correspondence between Gand the new graph G0 based on the level of abstraction f.
What is the first paper in the Reference?
0.8716846530357313
AnswerThe first paper in the reference is "Protein function prediction via graph kernels" by Karsten M Borgwardt, Cheng Soon Ong, Stefan Schonauer, SVN Vishwanathan, Alex J Smola, and Hans-Peter Kriegel.
Who are the authors of GraMMy?
0.9073243432001957
AnswerI don't know.
